# Пользовательские классы

## dataclass

На лекции мы видели пример создания класса `Point2D`. Это простой класс, который содержит два атрибута - абсциссу и ординату, и описывает точку двумерного пространства. Класс `Point2D` может быть описан так:

In [ ]:
class Point2D:
    abscissa: float
    ordinate: float

    def __init__(
        self,
        abscissa: float = 0.,
        ordinate: float = 0.,
    ) -> None:
        self.abscissa = abscissa
        self.ordinate = ordinate

На примере данного кода видно, что по умолчанию классы в Python обладают рядом недостатков. Один из таких недостатков - необходимость определять конструктор, даже если конструктор не обладает никакой сложной логикой. В данном примере при инициализации экземпляра `Point2D` в конструкторе не происходит никаких сложных вычислений. Мы просто принимаем данные от вызывающей стороны и сохраняем их в соответствующие атрибуты. И для реализации такой простой логики нам пришлось самим написать конструктор, который занимает больше половины определения класса `Point2D`. И это при том, что наш объект имеет всего два атрибута. Если бы атрибутов было бы больше, нам бы пришлось проделать еще больше утомительной работы для определения генерик конструктора.

Следующий минус - отсутствие читаемого строкового представления. Если мы попытаемся использовать экземпляр класса `Point2D` в качестве аргумента функции `print()`, то в стандартный поток вывода будет напечатана абра-кадабра. Использовать такой вывод в целях отладки или, тем более, для демонстрации пользователю крайне проблематично. 

In [ ]:
point1 = Point2D()
point2 = Point2D(abscissa=3.14, ordinate=2.72)

print(
    f"point1: {point1}",
    f"point2: {point2}",
    sep="\n",
)

Еще один минус заключается в логике сравнения экземпляров класса с помощью оператора `==`. В большинстве случаев под сравнением экземпляров класса с помощью оператора `==` удобно понимать сравнение с помощью оператора `==` значений соответствующих атрибутов экземпляров. Однако в Python по умолчанию сравнение экземпляров с помощью `==` происходит по `id`. Иными словами, по умолчанию результат сравнения экземпляров пользовательских классов с помощью оператора `==` соответствует результату сравнения этих же экземпляров с помощью оператора `is`. В большинстве задач это очень неудобно и нелогично.

In [ ]:
print(
    point1 == point2,
    point1 == Point2D(),
    sep="\n",
)

Чтобы избавиться от всех этих недостатков, в стандартной библиотеки Python существует модуль `dataclasses`. Ниже мы рассмотрим некоторые возможности данного модуля. Для детального ознакомления рекомендуем обратиться к [официальной документации](https://docs.python.org/3/library/dataclasses.html).

Основной объект модуля `dataclasses` - декоратор `dataclass`. Использование этого декоратора в момент определение класса позволяет определить простой `__init__` на основе "объявлени" атрибутов класса, добавить функционал для читаемого вывода экземпляров класса в `stdout`, а также определить адекватное сравнение экземпляров с помощью оператора `==`. 

In [ ]:
from dataclasses import dataclass


@dataclass
class Point2D:
    abscissa: float = 0.
    ordinate: float = 0.

In [ ]:
point1 = Point2D()
point2 = Point2D(abscissa=3.14, ordinate=2.72)

print(
    f"point1: {point1}",
    f"point2: {point2}",
    sep="\n",
)

In [ ]:
print(
    point1 == point2,
    Point2D() == point1,
    sep="\n",
)

При работе с `dataclass` могут возникнуть проблемы, если нам потребуется определить атрибут, который должен иметь некоторое значение по умолчанию изменяемого типа данных.

In [ ]:
try:
    @dataclass
    class ClassWithMutable:
        lst: list[str] = []

except ValueError as exc:
    print(f"error: {exc}")

В этом случае следует воспользоваться объектом `dataclasses.field`. Это специальный объект, который позволяет более подробно описывать поля класса. В частности, объект `field` имеет аргумент `default_factory`. С помощью этого аргумента можно определить фабрику для создания значений по умолчанию указанного атрибута.

In [ ]:
from dataclasses import field


@dataclass
class ClassWithMutable:
    lst: list[str] = field(default_factory=list)


print(ClassWithMutable())

Также полезно знать о функциях `asdict`, `astuple` и `fields`, т.к. с их помощью можно выполнять преобразования дата-классов в объекты других типов данных. Например, с помощью функции `asdict` можно сконструировать словарь на базе переданного дата-класса. Ключами словаря будут являться строки, соответствующие идентификаторам атрибутов экземпляра данного класса. Значения словаря - значения соответствующих атрибутов.

In [ ]:
from dataclasses import (
    asdict,
    astuple,
    fields,
)


@dataclass
class SegmentData:
    segment_id: str
    segment_type: str
    segment_start: float
    segment_end: float

In [ ]:
segment_data = SegmentData(
    segment_id="12345678",
    segment_type="voice",
    segment_start=0.0,
    segment_end=3.14,
)

print(
    f"dict: {asdict(segment_data)}",
    f"tuple: {astuple(segment_data)}",
    f"fields: {[field.name for field in fields(SegmentData)]}",
    sep="\n",
)

Подобные преобразования могут быть полезны в различных задачах.

## Задача

Представим, что мы занимаемся разработкой некоторого стримингового сервиса. Для того, чтобы пользователи получили возможность использовать наш сервис, они должно пройти регистрацию, придумав уникальный логин и надежный пароль. Также во время регистрации пользователи должны придумать себе никнейм, однако никнеймы не обязаны быть уникальными. После регистрации пользователь получает уникальный ID в формате UUID, а его данные сохраняются в базе данных. В рамках нашей задачи под базой данных будет подразумеваться некоторый словарь.

Однако, работать напрямую с базой данных не очень удобно. Поэтому для упрощения задач по работе с базой данных, мы решили создать обертку, которая реализует следующие операции:

- `create_person(person: Person) -> UUID` - создает новую запись о пользователе в базе данных. Прежде, чем создать запись о пользователе, происходит проверка логина и пароля. Логин должен быть уникальным и содержать только английские буквы в верхнем и нижнем регистре, а также цифры от 0 до 9. Логин не может быть пустой строкой. Также происходит проверка надежности пароля. Пароль считается надежным, если
  - пароль содержит хотя бы одну букву английского алфавита в верхнем регистре;
  - пароль содержит хотя бы одну букву английского алфавита в нижнем регистре;
  - пароль содержит хотя бы одну цифру от 0 до 9;
  - пароль состоит не менее чем из 10 символов;
  - пароль не содержит никаких символов, кроме разрешенных.  
  
  Если хотя бы одна проверка не проходит, обертка должна возбудить исключение `ValueError`. Иначе, создается новая запись в базе данных. Записи присваивается уникальный UUID, который возвращается в качестве результата вызывающей стороне. Это сделано, чтобы в дальнейшем вызывающая сторона могла манипулировать созданной записью по полученному UUID.

- `read_person(person_id: UUID) -> Person` - читает данные о пользователе из базы данных. На вход подается UUID пользователя. Если пользователя с полученным UUID нет в базе, возбуждается исключение `KeyError`. Иначе, метод читает данные о пользователе и возвращает их вызывающей стороне.

- `update_person(person_id: UUID, person_info_new: Person) -> None` - обновляет данные пользователя. Сначала происходит проверка переданного UUID. Если пользователя с переданным UUID нет в базе данных, возбуждается исключение `KeyError`. Если пользователь с переданным UUID есть в базе данных, происходит обновление полей записи. Новые значения берутся из аргумента `person_info_new`. Поле записи обновляется, но только если значение соответствующего ему поля в `person_info_new` - не пустая строка. Иначе поле остается без изменений. Если происходит обновление пароля или логина, необходимо осуществить их проверку по правилам, описанным выше, и возбудить исключение `ValueError`, если проверка не пройдена.

- `delete_person(person_id: UUID) -> None` - удаляет пользователя с переданным UUID из базы данных. Если пользователя с переданным UUID не было в базе данных, необходимо возбудить исключение `KeyError`.

Ваша задача - реализовать обертку для базы данных с описанным функционалом.

## Решение

In [ ]:
from dataclasses import dataclass, asdict
from uuid import (
    UUID,
    uuid4,
)

In [ ]:
@dataclass
class Person:
    """
    Информация о пользователе.

    Attrs:
        login: логин пользователя.
        password: пароль пользователя.
        username: имя пользователя.
        metadata: дополнительные сведения о пользователе.
    """

    login: str
    password: str
    username: str
    metadata: str = ""

In [ ]:
class PersonDB:
    _database: dict[UUID, Person]
    _login_registry: dict[str, str]

    def __init__(self) -> None:
        """Инициализирует базу данных."""
        # ваш код

    def create_person(self, person: Person) -> UUID:
        """
        Создает новую запись о пользователе в базе данных.

        Args:
            person: данные о пользователе, которые будут помещены в БД.

        Returns:
            UUID - идентификатор, который будет связан с созданной записью.

        Raises:
            ValueError, если логин или пароль не удовлетворяют требованиям.
        """
        # ваш код
    
    def read_person_info(self, person_id: UUID) -> Person:
        """
        Читает актуальные данные пользователя из базы данных.

        Args:
            person_id: идентификатор пользователя в формате UUID.

        Returns:
            Данные о пользователе, упакованные в структуру Person.

        Raises:
            KeyError, если в базе данных нет пользователя с person_id.
        """
        # ваш код
    
    def update_person_info(self, person_id: UUID, person_info_new: Person) -> None:
        """
        Обновляет данные о пользователе.

        Args:
            person_id: идентификатор пользователя в формате UUID.
            person_info_new: модель со значениями на обновление. Будут обновлены
                только те поля, чье значение отличается от пустой строки '',
                остальные поля будут оставлены без изменений.

        Raises:
            ValueError, если при обновлении логина или пароля логин или пароль
                не прошли этап валидации.
            KeyError, если в базе данных нет пользователя с person_id.
        """
        # ваш код
    
    def delete_person(self, person_id: UUID) -> None:
        """
        Удаляет запись о пользователе.

        Args:
            person_id: идентификатор пользователя в формате UUID.

        Raises:
            KeyError, если в базе данных нет пользователя с person_id.
        """
        # ваш код

## Проверки

### create_person

In [ ]:
person1 = Person(
    password="Aa1Bb2Cc3Dd4",
    login="login1",
    username="user#1",
)

database = PersonDB()
person1_id = database.create_person(person1)

assert len(database._database) == 1
assert len(database._login_registry) == 1
assert person1_id in database._database
assert person1.login in database._login_registry
assert database._database[person1_id] == person1

persons_wrong = {
    "no-login": Person(
        password="Aa1Bb2Cc3Dd4",
        login="",
        username="user#2",
    ),
    "existed-login": Person(
        password="Aa1Bb2Cc3Dd4",
        login="login1",
        username="user#2",
    ),
    "too-short-password": Person(
        password="12345",
        login="login2",
        username="user#2",
    ),
    "no-lower": Person(
        password="A1B2C3D4E5F",
        login="login2",
        username="user#2",
    ),
    "no-upper": Person(
        password="a1b2c3d4e5f",
        login="login2",
        username="user#2",
    ),
    "no-digits": Person(
        password="aAbBcCdDeEf",
        login="login2",
        username="user#2",
    ),
}

for test_name, wrong_person in persons_wrong.items():
    try:
        database.create_person(wrong_person)
        assert False, test_name

    except ValueError:
        assert True
        assert len(database._database) == 1
        assert len(database._login_registry) == 1

### read_person

In [ ]:
person = database.read_person_info(person1_id)
assert person1 == person
assert len(database._database) == 1
assert len(database._login_registry) == 1

try:
    fake_id = uuid4()
    person = database.read_person_info(fake_id)
    assert False

except KeyError:
    assert True
    assert len(database._database) == 1
    assert len(database._login_registry) == 1

### update_person

In [ ]:
person2 = Person(
    password="AaBbcC1234Dd",
    login="login2",
    username="user#2"
)
person2_id = database.create_person(person2)
assert len(database._database) == 2
assert len(database._login_registry) == 2
assert person2_id in database._database
assert person2.login in database._login_registry
assert database._database[person2_id] == person2

person2_updated = Person(
    password="abcDEF123456",
    login="LOGIN2",
    username="user#2",
)
person2_update = Person(
    password="abcDEF123456",
    login="LOGIN2",
    username="",
)

database.update_person_info(person2_id, person2_update)
assert len(database._database) == 2
assert len(database._login_registry) == 2
assert person2_id in database._database
assert person2.login not in database._login_registry
assert person2_updated.login in database._login_registry
assert database._database[person2_id] == person2_updated

### delete_person

In [ ]:
try:
    fake_id = uuid4()
    database.delete_person(fake_id)
    assert False

except KeyError:
    assert True
    assert len(database._database) == 2
    assert len(database._login_registry) == 2

database.delete_person(person2_id)
assert len(database._database) == 1
assert len(database._login_registry) == 1
assert person2_id not in database._database
assert person2_updated.login not in database._login_registry